In [2]:
import pandas as pd
import json
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import BertTokenizerFast, BertForSequenceClassification, Trainer, TrainingArguments

# Загрузка и подготовка данных
data = []
with open('105_first.json', 'r', encoding='utf-8') as file:
    for line in file:
        try:
            # Преобразование каждой строки в JSON объект
            json_obj = json.loads(line)
            if json_obj["text"] is None:
                continue
            data.append(json_obj)
        except json.JSONDecodeError:
            print("Ошибка чтения")
            continue

# Создание DataFrame из списка JSON объектов
df = pd.DataFrame(data)


# Объединяем все категории в один столбец для упрощения (пример)
df['category'] = df[['region', 'instance', 'name', 'caseNumber', 'entryDate', 'names', 'judge', 'resultDate', 'decision', 'endDate']].apply(lambda x: ','.join(x.dropna().astype(str)), axis=1)

# Токенизация текста
tokenizer = BertTokenizerFast.from_pretrained('bert-base-multilingual-cased')
tokenized_data = tokenizer(df['text'].tolist(), padding=True, truncation=True, max_length=65536, return_tensors="pt")

texts = df['text'].tolist()  # Список текстов
le = LabelEncoder()
df['labels'] = le.fit_transform(df['category'])
labels = df['labels'].tolist()

# Токенизация текстов
tokenized_texts = tokenizer(texts, padding=True, truncation=True, max_length=512, return_tensors="pt")

# Преобразование тензоров в списки для совместимости с train_test_split (если необходимо)
input_ids = tokenized_texts['input_ids'].tolist()  # Пример для PyTorch

# Теперь размерности должны совпадать
train_texts, test_texts, train_labels, test_labels = train_test_split(input_ids, labels, test_size=0.1)


In [3]:
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=len(le.classes_))

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_texts,  # Здесь необходимо подготовить данные в формате, пригодном для Trainer
    eval_dataset=test_texts     # Аналогично, подготовьте тестовые данные
)

trainer.train()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


TypeError: vars() argument must have __dict__ attribute

In [ ]:
results = trainer.evaluate()
print(results)


In [4]:
df['text']

0       ПРИГОВОР\nИМЕНЕМ РОССИЙСКОЙ ФЕДЕРАЦИИ\nс. Кыре...
1       ПРИГОВОР\nИМЕНЕМ РОССИЙСКОЙ ФЕДЕРАЦИИ\nс. Кыре...
2                                                    None
3       Дело №\nПРИГОВОР\nИМЕНЕМ РОССИЙСКОЙ ФЕДЕРАЦИИ\...
4       ПРИГОВОР\nИМЕНЕМ РОССИЙСКОЙ ФЕДЕРАЦИИ\nс. Кыре...
                              ...                        
2850    Уголовное дело № 1-629/2016\nП Р И Г О В О Р\n...
2851                                                 None
2852    \nУголовное дело №1-453-2012\nП Р И Г О В О Р\...
2853    Уголовное дело № 1-715/2011\nП Р И Г О В О Р\n...
2854    Уголовное дело № 1-600/ 2013\n\nП Р И Г О В О ...
Name: text, Length: 2855, dtype: object

In [9]:
tokenized_data

{'input_ids': tensor([[  101,   524, 18005,  ...,     0,     0,     0],
        [  101,   524, 18005,  ...,     0,     0,     0],
        [  101, 52935, 11602,  ...,     0,     0,     0],
        ...,
        [  101,   528, 85952,  ...,     0,     0,     0],
        [  101,   528, 85952,  ...,     0,     0,     0],
        [  101,   528, 85952,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [11]:
# Допустим, у вас есть DataFrame с текстами и метками
texts = df['text'].tolist()  # Список текстов
le = LabelEncoder()
df['labels'] = le.fit_transform(df['category'])
labels = df['labels'].tolist()

# Токенизация текстов
tokenized_texts = tokenizer(texts, padding=True, truncation=True, max_length=512, return_tensors="pt")

# Преобразование тензоров в списки для совместимости с train_test_split (если необходимо)
input_ids = tokenized_texts['input_ids'].tolist()  # Пример для PyTorch

# Теперь размерности должны совпадать
train_texts, test_texts, train_labels, test_labels = train_test_split(input_ids, labels, test_size=0.1)


In [13]:
!pip install accelerate -U

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 1.7 MB/s eta 0:00:00a 0:00:01


In [15]:
!pip install transformers[torch]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
zsh:1: no matches found: transformers[torch]
